In [1]:
import tushare as ts
import pandas as pd
import numpy as np
import statsmodels.api as sm
from statsmodels import regression
import ffn
from sklearn.svm import SVR
import matplotlib.pyplot as plt
from collections import defaultdict
import os

E:\Anaconda\lib\site-packages\statsmodels\compat\pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


In [57]:
class Model:
    
    #初始化参数
    def __init__(self, data, train_start, train_end, test_start, test_end, n):
        
        self.code = data['code']
        self.train_start = train_start
        self.train_end = train_end
        self.test_start = test_start
        self.test_end = test_end
        self.n = n
        
    #获取单只股票初始训练集和测试集
    def getInitData(self, code):
        
        train_start = self.train_start
        train_end = self.train_end
        
        test_start = self.test_start
        test_end = self.test_end
        
        train_data = ts.get_hist_data(code, start = train_start, end = train_end)
        test_data = ts.get_hist_data(code, start = test_start, end = test_end)

        train_data = train_data.loc[:,['open','high','close','low','price_change','p_change','turnover']]
        test_data = test_data.loc[:,['open','high','close','low','price_change','p_change','turnover']]
        
        return train_data, test_data
    
    #训练和预测股票的N期收益
    def train(self):
        #使用scikit-learn导入SVR进行训练和预测
        x_train = []
        y_label = []
        x_test = []
        
        for code in self.code:
            train_data, test_data = self.getInitData(code)
            
            #计算特征
            '''
            以后特征的计算，写在feature的class里即可
            '''
            feature = Feature(train_data, test_data, self.n)
            
            
            train_data, test_data = feature.cal_mean()
            train_data = feature.cal_nReturn(train_data)

            svr_rbf = SVR(kernel = 'rbf', C = 1e3, gamma = 0.1)
            X = train_data.iloc[:,-9:-2]
            X1 = test_data.iloc[:,-8:-1]
            y = train_data.iloc[:,-1:]
            for i in range(len(X)):
                x_train.append(list(X.iloc[i]))
            for i in range(len(y)):
                y_label.append(list(y.iloc[i]))
            for i in range(len(X1)):    
                x_test.append(list(X1.iloc[i]))

        y_rbf = svr_rbf.fit(x_train,y_label).predict(x_test)
        
        return y_rbf

    #得到预期代码

#     def getCode():
        
#         return 
        

In [58]:
class Feature:
    
    def __init__(self, train_data, test_data, n):
        self.train_data = train_data
        self.test_data = test_data
        self.n = n
        
    #计算开盘到收盘之间的价格均值
    def cal_mean(self):
        train_data = self.train_data
        test_data = self.test_data
        
        first_data_open = train_data.loc[:,['open']]
        first_data_close = train_data.loc[:,['close']]

        second_data_open = test_data.loc[:,['open']]
        second_data_close = test_data.loc[:,['close']]

        first_data_open.rename(columns = {'open':'mean'},inplace=True)
        first_data_close.rename(columns = {'close':'mean'},inplace=True)
        first_data_mean = (first_data_close + first_data_open)/2

        second_data_open.rename(columns = {'open':'mean'},inplace=True)
        second_data_close.rename(columns = {'close':'mean'},inplace=True)
        second_data_mean = (second_data_close + second_data_open)/2

        train_data['mean'] = first_data_mean
        test_data['mean'] = second_data_mean
        
        return train_data,test_data
    
    #计算N期收益
    def cal_nReturn(self, train_data):
        n = self.n
        #训练集的N期平均收益率
        origin_mean = train_data.loc[:,['mean']]
        origin_mean_n = origin_mean.shift(n)
        train_simpleret = (origin_mean - origin_mean_n) / origin_mean_n
        train_data['n_simpleret'] = train_simpleret
        #填充缺失的值
        train_data = train_data.fillna(method='bfill')
        
        return train_data

In [109]:
class Season_Feature:
    
    def __init__(self,year,season):
        self.year = year
        self.season = season
    
    
    
        
    #基本面数据的处理
    #
    #将基本面数据中的中文名称hash
    def name_to_code(name,data):
        d = defaultdict(list)
        real_name = []
        num = 0
        for i in name:
            if (not len(d[i])):
                d[i].append(num)
                num+=1
            real_name.append(d[i][0])
        text = str(num)
        return real_name
    
    #change_name 默认为基本面数据的列名，可以根据需要修改
    def need_change(data,change_name = ['name','industry','area']):
        for i in change_name:
            name = data[i]
            real_name = name_to_code(name,data)
            del data[i]
            data[i] = real_name
        return data
    
    #获取2017年第二季度的基本面数据
    def get_basic(self):
        year = self.year
        season = self.season
        
        base = ts.get_stock_basics()
        #获取2017年第2季度的业绩报表数据
        table1 = ts.get_report_data(year,season)
        #获取 2017年第二季度的盈利能力数据
        table2 = ts.get_profit_data(year,season)
        #获取2017年第2季度的营运能力数据
        table3 = ts.get_operation_data(year,season)
        #获取2017年第2季度的成长能力数据
        table4 = ts.get_growth_data(year,season)
        #获取2017年第2季度的偿债能力数据
        table5 = ts.get_debtpaying_data(year,season)
        #获取2017年第2季度的现金流量数据
        table6 = ts.get_cashflow_data(year,season)
        
        return base,table1,table2,table3,table4,table5,table6
    
    #将基本面的后6个表按 code，name 和并
    def merge_basic(self):
        year = self.year
        season = self.season
        
        base,table1,table2,table3,table4,table5,table6 = self.get_basic()
        #合并后6个表的数据
        result = pd.merge(table1,table2,on=['code','name'])
        result = pd.merge(result,table3,on=['code','name'])
        result = pd.merge(result,table4,on=['code','name'])
        result = pd.merge(result,table5,on=['code','name'])
        result = pd.merge(result,table6,on=['code','name'])
        #result = result.set_index('code')
        #合并前两个表的数据
        
        return result


In [60]:
#主函数
model = Model(ts.get_sz50s(), '2017-03-01', '2017-06-30', '2017-07-01', '2017-08-31', 15)

print (model.train())

[-0.01226435  0.00128179  0.00536256 ...,  0.05089279  0.04711263
  0.04966329]


In [110]:
'''
season2 = Season_Feature(2017,2)
season2 = season2.merge_basic()
season2 = season2.drop('name',1)
season2.to_csv('2017_season2.csv',sep='\t',na_rep='NaN',encoding='UTF-8')
'''


[Getting data:]########################################################[Getting data:]########################################################[Getting data:]########################################################[Getting data:]########################################################[Getting data:]########################################################[Getting data:]########################################################

In [107]:
'''
season1 = Season_Feature(2017,1)
season1 = season1.merge_basic()
season1 = season1.drop('name',1)
#season1 = season1.set_index('code')
season1.to_csv('2017_season1.csv',sep='\t',na_rep='NaN',encoding='UTF-8')
'''

[Getting data:]#######################################################[Getting data:]#######################################################[Getting data:]#######################################################[Getting data:]#######################################################[Getting data:]#######################################################[Getting data:]#######################################################

In [16]:
#load data
season1 = pd.read_csv('2017_season1.csv',sep='\t',index_col=0)
season2 = pd.read_csv('2017_season2.csv',sep='\t',index_col=0)

In [17]:
season1.head()

,code,eps_x,eps_yoy,bvps,roe_x,epcf,net_profits_x,profits_yoy,distrib,report_date,...,quickratio,cashratio,icratio,sheqratio,adratio,cf_sales,rateofreturn,cf_nm,cf_liabilities,cashflowratio
0,603129,0.30,87.50,NaN,7.36,NaN,2964.77,88.23,NaN,07-31,...,0.8655,65.6637,2004.3442,41.9311,58.0689,0.0942,0.0320,1.0771,0.0552,5.6048
1,300693,0.29,-17.14,NaN,7.56,NaN,1978.66,-17.37,NaN,07-31,...,1.8684,53.9644,2839.2049,56.2358,43.7642,-0.0335,-0.0091,-0.2240,-0.0209,-2.1422
2,603359,0.26,225.00,NaN,3.57,NaN,4439.76,208.22,NaN,07-24,...,1.2114,45.8946,12603.7017,52.0231,47.9769,0.4177,0.0410,2.2545,0.0854,8.5396
3,603978,0.35,6.06,NaN,3.16,NaN,2126.94,6.40,NaN,07-24,...,1.6079,24.9947,1854.3918,68.9945,31.0055,-0.0814,-0.0154,-0.7171,-0.0496,-5.7985
4,300687,0.25,NaN,NaN,5.55,NaN,1517.02,9.65,NaN,07-21,...,2.754,12.508,2982.8836,66.1234,33.8766,0.1848,0.0641,1.7946,0.1891,18.9367


In [18]:
#获取两个季度股票代码
def get_season_codes(season1,season2):
    season1_codes = season1['code']
    season2_codes = season2['code']
    return season1_codes,season2_codes

In [19]:
season1_codes,season2_codes = get_season_codes(season1,season2)

In [75]:
test = season1.code
test

0       603129
1       300693
2       603359
3       603978
4       300687
5       603860
6       300686
7       603387
8       300682
9       300682
10      603357
11      300681
12      300680
13      603233
14        2888
15      603617
16      300676
17        2887
18      603938
19         511
20      603595
21      300672
22      603757
23      603316
24      603933
25      603331
26      603326
27      603326
28      300671
29      300670
         ...  
3216    600017
3217      2534
3218    300484
3219    601678
3220      2415
3221    601118
3222    601118
3223    600829
3224    300286
3225       619
3226    600436
3227      2296
3228    300493
3229      2732
3230    300018
3231      2536
3232    603333
3233    603333
3234    603797
3235    600123
3236       967
3237       953
3238      2553
3239      2686
3240    600966
3241      2237
3242      2664
3243    300310
3244    300101
3245    300294
Name: code, Length: 3246, dtype: int64

In [84]:
#有些股票的第一季度历史数据为空， 为空则抛弃
data = ts.get_hist_data(str(test[3240]),start='2017-01-01',end='2017-03-31')



In [100]:
data.empty

if type(data)!='NoneType':
    print(1)

1


In [36]:
#获取时间序列
season_date = data.index.tolist()
print(len(season_date))
data_open = data.open
data_open = list(data_open)
data_open[1]
print(type(data_open[1]))

59
<class 'numpy.float64'>


In [ ]:
#获取季度内股票的历史数据 返回的是股票代码和收益率的label
def get_season_hist_data(season_codes,season_start='2017-01-01',season_end='2017-03-31'):
    y_label = []
    x_code = []
    for code in season_codes:
        stock = str(code)
        if len(stock)<6:
            continue
        data = ts.get_hist_data(stock,season_start,season_end)
        #需要考虑在当前季度中没有该股票的历史数据的情况
        #没有数据无法进行计算，直接抛弃。
       
        if (type(data) == 'NoneType'):
            continue
        elif data.empty == True:
            continue
        else:
            #获取需要计算收益的两列数据
            data_open = data.open
            data_open = list(data_open)
            data_close = data.close
            data_close = list(data_close)
            # 获取季度有交易日区间
            days = len(data.index.tolist())
        
            #找出收益率 >=15% 和 收益率 <= -5%的股票并给出对应的y_label
            
            ret_flag = 0 #收益率的标记 为0表示在该季度的交易日内没能达到要求
            
            for i in range(0,days):
                ret_flag = 0
                for j in range(i+1,days):
                #计算收益率
                #只要在季度交易区间内达到了限制条件，就设置相应的label 在交易期间没达到就按照季初-季末算收益率
                    ret = (data_close[j] - data_open[i]) / data_close[j]
        
                    if ret >= 0.15 :
                        x_code.append(code)
                        y_label.append(0.15) 
                        ret_flag = 1
                        break
                    elif ret <= -0.05:
                        x_code.append(code)
                        y_label.append(0.15) 
                        ret_flag = 1
                        break
                    if ret_flag == 1:
                        break
            #若没有达到要求，将label 设置成季初到季末的收益率
            if ret_flag == 0:
                x_code.append(code)
                ret = (data_close[days] - data_open[0]) / data_close[days]
                y_label.append(ret) 
               
                   
    return x_code,y_label
            
        
        

In [ ]:
x_code,y_label = get_season_hist_data(season1_codes,'2017-01-01','2017-03-31')